In [ ]:
# Install required packages if not already installed
# !pip install flask pandas scikit-learn --quiet

import os
import ast
import pandas as pd
from flask import Flask, request, render_template_string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load and preprocess data
def load_data():
    data_path = "RAW_recipes.csv"  # Ensure this file is in the same folder as your script

    if not os.path.exists(data_path):
        print(f"Error: File not found at {data_path}")
        return pd.DataFrame()
    
    df = pd.read_csv(data_path, encoding="utf-8")
    
    # Basic data cleaning
    df["name"] = df["name"].fillna("")
    df["ingredients"] = df["ingredients"].fillna("[]").apply(
        lambda x: ' '.join(ast.literal_eval(x)) if isinstance(x, str) else "")
    df["tags"] = df["tags"].fillna("[]").apply(
        lambda x: ' '.join(ast.literal_eval(x)) if isinstance(x, str) else "")
    
    df["features"] = df["name"] + " " + df["ingredients"] + " " + df["tags"]
    return df

# Load dataset
df = load_data()
if df.empty:
    raise SystemExit("Dataset could not be loaded. Please ensure RAW_recipes.csv is in the current folder.")

# TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(df["features"])

# Recommendation logic
def recommend_food(user_input, top_n=5):
    user_vec = vectorizer.transform([user_input])
    similarities = cosine_similarity(user_vec, tfidf_matrix)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices][["name", "minutes", "tags"]]

# Flask app setup
app = Flask(__name__)

# HTML template
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Food Recommender</title>
</head>
<body>
    <h2>Food Recommendation System</h2>
    <form method="POST">
        <label for="preferences">Enter your food preferences (e.g., spicy chicken garlic):</label><br><br>
        <input type="text" id="preferences" name="preferences" size="50" required>
        <input type="submit" value="Recommend">
    </form>
    {% if recommendations %}
        <h3>Top Recommendations:</h3>
        <ul>
        {% for item in recommendations %}
            <li><strong>{{ item.name }}</strong> - {{ item.minutes }} minutes<br>
            Tags: {{ item.tags }}</li>
        {% endfor %}
        </ul>
    {% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    recommendations = []
    if request.method == "POST":
        user_input = request.form["preferences"]
        if user_input.strip():
            recommendations = recommend_food(user_input).to_dict(orient="records")
    return render_template_string(HTML_TEMPLATE, recommendations=recommendations)

if __name__ == "__main__":
    # Start Flask server
    app.run(host="127.0.0.1", port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/May/2025 22:38:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2025 22:38:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/May/2025 22:38:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2025 22:42:10] "POST / HTTP/1.1" 200 -
